In [1]:
import pandas as pd
import numpy as np
import re

import csv

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import spacy

import json

In [2]:
from nltk.corpus import brown
#brown.words()

In [3]:
fuzz.partial_ratio("this is a test", "this is a test!")

100

In [4]:
route = '../../../data/'

# Data Supermercados

In [5]:
df_clean = pd.read_csv(route + 'datamarket_clean_unique.csv')
#df_clean.head()

In [6]:
#df_clean.info()

In [7]:
df_carrefour = df_clean[df_clean['supermarket']=='carrefour-es']
df_mercadona = df_clean[df_clean['supermarket']=='mercadona-es']
df_dia = df_clean[df_clean['supermarket']=='dia-es']

# Receta de cocina a analizar

In [8]:
# Comprobar ingrediente a ingrediente si ya existen en las listas intermedias

receta_a_analizar = '''200 g de zanahoria troceada
2 dientes de ajo
1 - 2 ramitas de perejil fresco (sólo las hojas)
50 g de pan rallado
2 huevos
50 g de leche
1 cucharadita de sal
1 pellizco de nuez moscada molida
1 pellizco de canela molida
1 pellizco de pimienta negra molida
500 g de carne picada mixta (250 g de ternera y 250 g de cerdo)
50 g de piñones (opcional)
20 g de aceite de oliva virgen extra (para engrasar la bandeja)
harina (para rebozar)
250 g de cebolla cortada en cuartos
2 dientes de ajo
50 g de aceite de oliva virgen extra
1 cucharada de tomate concentrado
100 g de vino blanco
300 g de agua
1 pellizco de sal
1 pellizco nuez moscada molida
1 pellizco de pimienta negra molida
1 cucharada de extracto de buey
1 pellizco de clavo molido
1 cucharada de perejil fresco picado'''

In [9]:
separador = '\n'

In [10]:
receta_a_analizar_lst = receta_a_analizar.split(separador)
receta_a_analizar_lst

['200 g de zanahoria troceada',
 '2 dientes de ajo',
 '1 - 2 ramitas de perejil fresco (sólo las hojas)',
 '50 g de pan rallado',
 '2 huevos',
 '50 g de leche',
 '1 cucharadita de sal',
 '1 pellizco de nuez moscada molida',
 '1 pellizco de canela molida',
 '1 pellizco de pimienta negra molida',
 '500 g de carne picada mixta (250 g de ternera y 250 g de cerdo)',
 '50 g de piñones (opcional)',
 '20 g de aceite de oliva virgen extra (para engrasar la bandeja)',
 'harina (para rebozar)',
 '250 g de cebolla cortada en cuartos',
 '2 dientes de ajo',
 '50 g de aceite de oliva virgen extra',
 '1 cucharada de tomate concentrado',
 '100 g de vino blanco',
 '300 g de agua',
 '1 pellizco de sal',
 '1 pellizco nuez moscada molida',
 '1 pellizco de pimienta negra molida',
 '1 cucharada de extracto de buey',
 '1 pellizco de clavo molido',
 '1 cucharada de perejil fresco picado']

# Creación y alimentación de tablas de productos intermedias

In [11]:
# Opening JSON file
with open(route + 'data_mercadona.json') as json_file:
    a_mercadona = json.load(json_file)
#a_mercadona

In [12]:
with open(route + 'data_dia.json') as json_file:
    a_dia = json.load(json_file)
#a_dia

In [13]:
with open(route + 'data_carrefour.json') as json_file:
    a_carrefour = json.load(json_file)
#a_carrefour

# Localizar ingredientes en Lista intermedia

In [14]:
nlp = spacy.load('es_core_news_sm')

In [15]:
def normalize_lemma_space(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 2 and t.isalpha()]
    cadena = " ".join(lexical_tokens)
    doc2 = nlp(cadena)
    lemmas = [tok.lemma_.lower() for tok in doc2 if tok.pos_ != 'PRON']
    lemmas_cadena = " ".join(lemmas)
    return lemmas_cadena

In [16]:
def get_key_generico(ingrediente, lista_supermercado):
    # list out keys and values separately
    key_list = list(lista_supermercado.keys())
    val_list = list(lista_supermercado.values())
    
    # lematizamos ingrediente primero
    ingrediente_lemma = normalize_lemma_space(ingrediente)
    
    # hacemos fuzzybuzzy sobre la lista intermedia
    ratios = process.extractOne(ingrediente_lemma, lista_supermercado)
    if int(ratios[1]) > 75: # can play with this number 
        position = val_list.index(ratios[0])
        return [key_list[position], ratios[0], ratios[1]]
 
    return ['No product_ID', 'No product Match', 'No product Match']

### Localizar todos los ingredientes de la receta

In [ ]:
for i in receta_a_analizar_lst:
    print(get_key_generico(i, a_mercadona), i)

### Localizar un único ingrediente

In [ ]:
ingrediente_validar = '1 pellizco de clavo molido'

In [ ]:
#Al ser todas las tablas intermedias iguales, miramos en una en concreto:
print(normalize_lemma_space(ingrediente_validar))

get_key_generico(ingrediente_validar, a_dia)

### Dia

In [17]:
df_dia.reset_index(drop=True, inplace=True)

In [25]:
df_dia.loc[(df_dia['name'].str.lower().str.contains('piñon', na=False)),
           #& (df_dia['category'].str.lower().str.contains('lacteos_y_huevos_leche_entera', na=False)),
                 ['name','category', 'product_id', 'price', 'reference_price', 'reference_unit','Date', 'url']]

,name,category,product_id,price,reference_price,reference_unit,Date,url
746,DIA crema de champiñones sobre 68 gr,despensa_sopas_cremas_y_pures,91cd20e7e1f5281dcb12e7aeb1878f1f,0.55,8.09,kg,2021-12-02,https://www.dia.es/compra-online/despensa/sopa...
855,DIA SELECCIÓN FRESCA Redondo de vacuno con pas...,frescos_carne_otras_aves,aedd37cb225de8ae8ae4a92699ee99c2,7.49,9.99,kg,2022-01-05,https://www.dia.es/compra-online/frescos/carne...
882,DIA SELECCIÓN FRESCA Redondo de cerdo con pimi...,frescos_carne_cordero,1a624919fc848fd66826d6c32b329e7b,6.99,9.32,kg,2022-01-11,https://www.dia.es/compra-online/frescos/carne...
1348,GALLO piñones paquete 450 gr,despensa_pastas_harinas_y_masas_pastas,b813fd73ad8002b93508e6136632afce,1.33,2.96,kg,2022-05-27,https://www.dia.es/compra-online/despensa/past...
2174,DIA NATURMUNDO piñones al natural bolsa 70 gr,despensa_aperitivos,b9e3e470665a1813895ab15bf84c12ab,4.25,60.71,kg,2022-11-03,https://www.dia.es/compra-online/despensa/aper...
2899,DIA VEGECAMPO champiñones enteros lata 185 gr,despensa_conservas_conservas_vegetales,ebdcf2d9c9a0f58ba2ef1c7d2301490d,1.19,6.43,kg,2022-11-04,https://www.dia.es/compra-online/despensa/cons...
2934,DIA VEGECAMPO champiñones laminados lata 185 gr,despensa_conservas_conservas_vegetales,e5f7537650c4d6285fb53e0f74f3ce6d,0.90,4.86,kg,2022-11-04,https://www.dia.es/compra-online/despensa/cons...
3816,DIA AL PUNTO salteado de setas y champiñones b...,congelados_precocinados,ad04edaa5bc89b038a1d340e4a52c093,1.85,4.11,kg,2022-11-04,https://www.dia.es/compra-online/congelados/pr...


In [26]:
df_dia.iloc[1348]['url']

'https://www.dia.es/compra-online/despensa/pastas-harinas-y-masas/pastas/p/279611'

In [ ]:
# Añadimos nuevos elementos a la lista intermedia

a_dia['5bfb7f83d904198604a60ed822b8a61b'] = 'clavo molido'
#a_dia

### Carrefour

In [19]:
df_carrefour.reset_index(drop=True, inplace=True)

In [41]:
df_carrefour.loc[(df_carrefour['name'].str.lower().str.contains('piñon', na=False)),
                 #& (df_carrefour['category'].str.lower().str.contains('el_mercado_verduras_y_hortalizas_tomates_y_pep', na=False)),
                 ['name','category', 'product_id', 'price', 'reference_price', 'reference_unit','Date', 'url']]

,name,category,product_id,price,reference_price,reference_unit,Date,url
746,Hamburguesa vegetal de champiñones Suquipá 180 g,el_mercado_carniceria_hamburguesas,d488a137436e5d8e70b9a5a3cd572fe0,2.25,12.50,kg,2021-04-02,https://www.carrefour.es/supermercado/hamburgu...
1096,Crema de setas del bosque con champiñones Knor...,la_despensa_conservas_sopas_y_precocinados_cal...,c3561f3ae6dd7fa0cc1f457e5bb1132a,2.00,4.00,l,2021-04-06,https://www.carrefour.es/supermercado/crema-de...
1136,Pollo con champiñones Carretilla 250 g.,la_despensa_conservas_sopas_y_precocinados_pla...,55b7a2d1669606cf7495a15ae70673c4,2.99,11.96,kg,2021-04-06,https://www.carrefour.es/supermercado/pollo-co...
2425,"Capón relleno asado con cerdo, pepitas de mous...",productos_frescos_carniceria_otras_carnes,5ac5927dda9f5b5b8a99d3a1a4f54f92,74.97,29.99,kg,2021-12-03,https://www.carrefour.es/supermercado/capon-re...
3211,Crema de champiñones Gallina Blanca 57 g.,la_despensa_conservas_sopas_y_precocinados_cal...,5d5876672e80f8cae90b10b3a3d6b870,1.14,20.00,kg,2022-04-15,https://www.carrefour.es/supermercado/crema-de...
4339,Champiñon cuartos Carrefour bandeja 300 g aprox,productos_frescos_verduras_y_hortalizas_setas_...,1dd73a589ed4b0cead206af2c068c0dd,1.80,6.00,kg,2022-07-26,https://www.carrefour.es/supermercado/champino...
4497,Champiñon pie cortado Carrefour 400 g aprox,productos_frescos_verduras_y_hortalizas_setas_...,0d2136808867357e9282a684facfdb5f,1.60,3.99,kg,2022-08-05,https://www.carrefour.es/supermercado/champino...
4845,Espinacas con pasas y piñones Findus 400 g.,la_despensa_conservas_sopas_y_precocinados_sal...,0a6a7204e32d80a7e850ac253d479dea,2.89,7.22,kg,2022-09-07,https://www.carrefour.es/supermercado/espinaca...
6330,Champiñones laminados Carrefour 230 g.,la_despensa_conservas_sopas_y_precocinados_con...,1f63800c444584f7ffe12265ccbf805d,1.06,4.61,kg,2022-11-02,https://www.carrefour.es/supermercado/champino...
6333,Champiñones laminados Carrefour pack de 3 unid...,la_despensa_conservas_sopas_y_precocinados_con...,0f2d7e5309845634af03711a07c6c1b9,1.58,4.58,kg,2022-11-02,https://www.carrefour.es/supermercado/champino...


In [ ]:
df_carrefour.iloc[6952]['url']

In [ ]:
# Añadimos nuevos elementos a la lista intermedia

a_carrefour['839abd82070887655436dbda5c9aeed0'] = 'clavo molido'
#a_carrefour

### Mercadona

In [33]:
df_mercadona.reset_index(drop=True, inplace=True)

In [34]:
df_mercadona.loc[(df_mercadona['name'].str.lower().str.contains('piñon', na=False)),
                 ['name','category', 'product_id', 'price', 'reference_price', 'reference_unit','Date', 'url']]

,name,category,product_id,price,reference_price,reference_unit,Date,url
497,Crema de queso con tocino de cielo y piñones R...,postres_y_yogures_gelatina_y_otros_postres,e5ce68c52e36aa6a1e7ee7ce565fdd3f,1.30,7.223,kg,2021-10-26,https://tienda.mercadona.es/product/68333/crem...
544,Salsa champiñones Hacendado,aceite_especias_y_salsas_otras_salsas,1865116f47c0ca3d1ce59dc54dc9d645,0.80,2.581,100g,2021-11-15,https://tienda.mercadona.es/product/17508/sals...
788,Crema de champiñones Hacendado,conservas_caldos_y_cremas_gazpacho_y_cremas,fb45d2e47bed3b5501d916faaddf126f,0.48,7.385,kg,2022-02-06,https://tienda.mercadona.es/product/8711/crema...
1183,"Pizza Tirolesa con speck, champiñones y brie H...",congelados_pizzas,4da39c89ac28fab00b489176aec35470,2.60,6.934,kg,2022-05-26,https://tienda.mercadona.es/product/63662/pizz...
1534,Piñones Hacendado,aceite_especias_y_salsas_especias,e843cf814a8869427b16e78cd9e3a27e,2.65,53.000,kg,2022-09-06,https://tienda.mercadona.es/product/22206/pino...
2276,Piñones ibéricos Hacendado,aceite_especias_y_salsas_especias,389b950cbbed922cf39ea931230d136e,5.50,110.000,kg,2022-11-04,https://tienda.mercadona.es/product/34210/pino...
2305,Piñones Hacendado,aceite_especias_y_salsas_especias,115588048e3d76e8acf769fa3d498bff,2.95,42.143,kg,2022-11-04,https://tienda.mercadona.es/product/60869/pino...
3265,Piñones Hacendado,aperitivos_frutos_secos_y_fruta_desecada,b4c1741a70711a4e3c00fb59316b4f80,1.85,18.500,kg,2022-11-04,https://tienda.mercadona.es/product/36705/pino...
3344,Rellenito de vacuno con pasas y piñones,carne_vacuno,763b1be3132846b55a009f8e6375d806,11.95,11.950,kg,2022-11-04,https://tienda.mercadona.es/product/3555/relle...
4175,Crema de setas del bosque Knorr con champiñones,conservas_caldos_y_cremas_gazpacho_y_cremas,349cba5a04ef7f5024ac7381593ba2ee,2.25,4.500,l,2022-11-04,https://tienda.mercadona.es/product/8611/crema...


In [37]:
df_mercadona.iloc[2305]['url']

'https://tienda.mercadona.es/product/60869/pinones-hacendado-bote'

In [47]:
# Añadimos más elementos a lista intermedia

a_mercadona['da7ef546df8e21a55873dcd82c560902'] = 'nuez moscado molido'
#a_mercadona

# Actualización de listas intermedias de productos fuera de fecha

In [ ]:
# IDs a actualizar









In [ ]:
# Localizar en mi lista intermedia el producto a actualizar en base al ID

print(a_carrefour['cd2697f65f9695fcb5596161aa4c5cfa'])

In [48]:
a_mercadona

{'ecb81249f1dea9150ee04ed7a178029e': 'lenteja',
 'c71150390be4bd6853c9449a8cff62eb': 'pimiento verde',
 '43c81c10c1d9eb52b2410de54f274c9d': 'pimiento rojo',
 'c0ac3da899e51308db2492c91aebbcb2': 'zanahoria',
 'b2b8c39d71440a4cba62b86f31ce15e7': 'cebolletar',
 '2f4a5c7049e822ce171768cde7bd2e76': 'cebolla',
 '488e146ef0d8864d12b7ea721d8de22a': 'aceite olivo virgen extra',
 'e4b0fed91e0f81060e98ed076ebe39e4': 'aceite olivo',
 '14efd535d750388baa8122354a3bf2ff': 'sal',
 '370579b0cb90786d050efc5026103a3f': 'pimentón',
 '74270a2e828834e1dc65088ac8947be5': 'pastilla de caldo de pollo',
 '9161c686d13e241ab7af3f7ef1c191b0': 'caldo',
 '4cb684b2f6a44a7d56f7d49b0a265f17': 'chorizo',
 'f06a69f6ac8f49bfb052af3e0f614998': 'jamón curado dado',
 'c1e1a4c9d0af6984f9c1da705b025be1': 'agua',
 'fd8b87c9d8cd9c6c80cad59278ee37f3': 'diente ajo',
 '32d27a8b219ef9776eea1b48ee66f0cb': 'tomate troceado conserva',
 '2110b3edfbc5aed2a0f8e8525095dad7': 'pimiento piquillo conserva',
 'bf205f68e2169ec994522c3c5d93070f'

In [42]:
# Cambiar una Key de un diccionario primero va 'new_key' luego 'old_key'

a_carrefour['115588048e3d76e8acf769fa3d498bff'] = a_carrefour['d0ed6376e0508443e3c5f9ef48155e48']
del a_carrefour['d0ed6376e0508443e3c5f9ef48155e48']

a_carrefour

{'7d751158fe2567a81e991a0a572ce83f': 'lenteja',
 'c86eacc22285ad2f5e907acce7cdb5c7': 'pimiento rojo',
 '53e9f777e840fd172ee8bae9747d61f6': 'zanahoria',
 '452cf4289710f6cb5edb3607fc646185': 'cebolletar',
 '9d368b1cf54fbb93051b576c09753663': 'cebolla',
 'ec7cf862264cec9b1bc8344574840325': 'aceite olivo virgen extra',
 '276a2380dabc5045fcd8a869f60b88fc': 'aceite olivo',
 '5cd6ebfcd722da53b2e3dbdc7a535530': 'sal',
 'd4b324b3a39f8ffcd7bf37cc1a98b38c': 'pimentón',
 'd5430323125823fe208a1977a4e8e6fc': 'pastilla de caldo de pollo',
 'c33ea2b6f4582b4e4c0579520bb41bcc': 'caldo',
 '411367289ea7cb6032db7d65f7286bf4': 'chorizo',
 '87074ba46a16a1e75e8e82ce6a0ec154': 'jamón curado dado',
 '97dc915b2efecc4be9db3d93635c4c19': 'agua',
 'e85fa0fa9deef59040e440a058b676a0': 'diente ajo',
 '3a5a96c8fb4489d94de742abfc9f40e1': 'tomate troceado conserva',
 '004d6b13bdbfa5ba4945c9d645de3af9': 'pimiento piquillo conserva',
 '82ede0c9dd536b0d2d369e2e9a8e6fb2': 'azúcar',
 '127a3fc17b8bac1dfdb47ed6438d9e30': 'pimie

In [ ]:
# Cambiar un value de un diccionario

#a_carrefour['f02a315ad24d97cf459a7b0a6ee2ee9b'] = 'caldo pescado'
#a_dia['20ab76a0c057671b7d848bbda6551965'] = 'judía verde frasco'
#a_mercadona['e42a13f246bdd6a0b266ba06bd3b795b'] = 'gamba crudo'

### Eliminar espacios del principio y del final en los valores

In [ ]:
# Esto no haría falta en un principio puesto que ya parece que los espacios no se suceden

#a_mercadona = {k: v.strip() if isinstance(v, str) else v for k, v in a_mercadona.items()}
#a_carrefour = {k: v.strip() if isinstance(v, str) else v for k, v in a_carrefour.items()}
#a_dia = {k: v.strip() if isinstance(v, str) else v for k, v in a_dia.items()}

In [ ]:
#a_dia

# Dic to JSON

In [49]:
with open(route + 'data_mercadona.json', 'w') as f:
    json.dump(a_mercadona, f)

In [44]:
with open(route + 'data_dia.json', 'w') as f:
    json.dump(a_dia, f)

In [45]:
with open(route + 'data_carrefour.json', 'w') as f:
    json.dump(a_carrefour, f)

In [ ]:
a_mercadona

In [ ]:
a_dia

In [ ]:
a_carrefour